<a href="https://colab.research.google.com/github/skymined/study_data_analytics/blob/main/docs/quests/CDAs/CDA_categorical_continuous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setting

In [38]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 코렙의 경우 설치 후 runtime  > restart 해야함

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic')

## X(연속형) / Y(연속형) 케이스
- 연속 변수에 대한 분석은 종종 상관 관계, 회귀 또는 평균 비교에 중점

### 5. 고객 충성도 점수(X)가 평균 거래 가치(Y)를 예측합니까?
- 분석 방법: 선형 회귀를 사용하여 충성도 점수가 거래 가치를 얼마나 잘 예측하는지 결정합니다. 비선형 관계를 포착하기 위해 상호작용 항이나 다항 회귀를 포함할 수도 있습니다.


#### 분석 과정
- 고객 충성도 점수 : 고객의 구매 빈도수로 생각
- 평균 거래 가치 : 고객의 구매 총액으로 생각
- 귀무가설 : 자주 구매하는 고객은 수익 총액에 영향을 미치지 않는다.
- 대립가설 : 자주 구매하는 고객은 수익 총액에 영향을 미친다.


In [68]:
df_fruit = pd.read_csv("fruitstore_saleshistory.csv")
df_fruit[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05


In [69]:
df_fruit['회원번호'].value_counts()

1670400000    3586
436681536     1362
444617792     1166
548171456     1155
72796032      1120
              ... 
1160536384       1
401603136        1
1579504256       1
1202006848       1
26440576         1
Name: 회원번호, Length: 11202, dtype: int64

In [70]:
df_fruit['구매횟수'] = df_fruit['회원번호'].map(df_fruit['회원번호'].value_counts())

In [71]:
df_fruit[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,...,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각,구매횟수
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),...,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04,343
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),...,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05,60


In [73]:
df_fruit['구매총액'] = df_fruit.groupby('회원번호')['구매금액'].transform('sum')
df_fruit[:3]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,...,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각,구매횟수,구매총액
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),...,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04,343,3668670
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),...,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05,60,443111
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),...,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05,60,443111


#### 각 데이터들의 정규분포 여부 확인
- 구매 횟수 : df_fruit['구매횟수']
- 구매 총액 : df_fruit['구매총액']

In [75]:
df_fruit_id = df_fruit.drop_duplicates('회원번호')
df_fruit_id['회원번호'].value_counts()

272369856     1
1274964352    1
391666112     1
403067520     1
936611840     1
             ..
709585920     1
748927552     1
1263483136    1
172457664     1
26440576      1
Name: 회원번호, Length: 11202, dtype: int64

In [76]:
import scipy.stats as stats
stats.anderson(df_fruit_id['구매횟수'])
# p = 0.787 > 0.05 이므로 정규분포

AndersonResult(statistic=1383.1948793205702, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=62.98616318514551, scale=109.8922477723606)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

In [77]:
stats.anderson(df_fruit_id['구매총액'])
# p = 0.787 > 0.05 이므로 정규분포

AndersonResult(statistic=1477.9825381982573, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=687811.3006605963, scale=1264796.2039800137)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

#### 둘 다 정규 분포 이므로 pearsonr 사용

In [78]:
stats.pearsonr(df_fruit_id['구매횟수'], df_fruit_id['구매총액'])

PearsonRResult(statistic=0.9598828995945157, pvalue=0.0)

#### 상관관계 확인

In [79]:
df_fruit_id[['구매횟수', '구매총액']].corr()

,구매횟수,구매총액
구매횟수,1.000000,0.959883
구매총액,0.959883,1.000000


#### 분석결과 (기준 p < 0.05)
- 통계 측면 결론 : 귀무가설 거짓, 대립가설 참
- 고객의 충성도는 평균 거래 가치와 연관이 있다.
- 상관관계가 1에 근접하고 있으므로 고객의 충성도가 높음에 따라(자주 구매함에 따라) 평균 거래 가치가 증가함을 알 수 있다.